In [1]:
import numpy as np
import matplotlib as plt
import random

import keras
(train_inputs, train_labels), (test_inputs, test_labels) = keras.datasets.mnist.load_data()

train_inputs = train_inputs.astype("float32") / 255.0
test_inputs = test_inputs.astype("float32") / 255.0

training_data = [np.array(x.flatten()) for x in train_inputs]
test_data = [np.array(x.flatten()) for x in test_inputs]

2024-07-19 09:56:26.957971: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def loss(yhat, y):
    m = y.shape[0]
    epsilon = 1e-7
    yhat = np.clip(yhat, epsilon, 1 - epsilon)
    losses = -np.sum(y * np.log(yhat))
    return losses/m

In [3]:
label = np.eye(10)[train_labels]

In [4]:
def fitness(loss_list):
    return (((loss_list - loss_list.min()) / (loss_list.max() - loss_list.min())) - 0.5).flatten()

In [5]:
class oneLayerNN(object):
    def __init__(self, layer):
        self.layer = layer
        self.weight = np.random.rand(layer[0],layer[1]) * 0.001
        self.w_len = len(self.weight)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / np.sum(exp_x, keepdims=True)
    
    def forward(self, a):
        return self.softmax(np.dot(a, self.weight))
    
    def nevo(self):
        sigma = 1e-3
        theta = self.weight.ravel()
        # print(self.weight)
        for generation in range(100):
            directions = []
            loss_list = []
            accuracy = []
            for individual in range(2500):
                # directions = []
                epsilon =  np.random.normal(0, sigma, size=theta.shape)
                losses = []
                for batch in range(15):
                    # dd = []
                    idx = np.random.randint(0, len(training_data) - 1)
                    sample = training_data[idx]
                    self.weight = (theta + epsilon).reshape(self.weight.shape)

                    pred = self.forward(sample)
                    this_loss = loss(pred, label[idx])
                    
                    losses.append(this_loss)
                    accuracy.append(np.argmax(pred) == train_labels[idx])
            
                directions.append(epsilon)
                loss_list.append(np.mean(losses))

            
            directions = np.array(directions)
            loss_list = np.array(loss_list)
            
            # normalize to -0.5 to 0.5
            fitness = ((loss_list - loss_list.min()) / (loss_list.max() - loss_list.min())) - 0.5
            # fitness = fitness.flatten()

            # estimate gradient by multiplying the random directions with the found loss decrease
            #  note that this includes the directions that turned out to increase the loss, but
            #  since the values are normalized between -0.5 and +0.5, they will end up with a 
            #  negative sign so everything works out
            gradient = np.mean(np.multiply(directions.T, fitness), axis =1) / sigma
            # bonus question: why devide by sigma? 

            # use the gradient for gradient descent
            theta = theta - 0.1 * gradient

            # print(len(theta))

            print(f"Iteration: {generation+1}/100, mean accuracy: {np.mean(accuracy):.4f} \n")



In [8]:
ls = oneLayerNN([784, 10])
ls.forward(training_data[1])
# print(train_labels[1])
# ls.nevo()

array([0.10015227, 0.10047147, 0.10042998, 0.09989981, 0.09999721,
       0.09972166, 0.10002837, 0.09957569, 0.09993578, 0.09978774])